In [ ]:
from googleapiclient.discovery import build 
import pandas as pd 
from dotenv import load_dotenv
import os
import re
load_dotenv()
api_key =  os.getenv("API_KEY")
channel_id = "UCWsDFcIhY2DBi3GB5uykGXA"

youtube = build('youtube', 'v3', developerKey=api_key)
Get Channel Data
def get_channel_stats(youtube, channel_id):
    
    request = youtube.channels().list(
        part="snippet,statistics",
        id=channel_id
    )
    response = request.execute()

    data =  dict(Channel_name = response['items'][0]['snippet']['title'], 
                 Subscribers = response['items'][0]['statistics']['subscriberCount'],
                 Views = response['items'][0]['statistics']['viewCount'],
                 Total_videos = response['items'][0]['statistics']['videoCount']
                 )
    
    return data
get_channel_stats(youtube, channel_id)
Get Video Data
#
def get_channel_videos(channel_id):

    channel_response = youtube.channels().list(
        part='contentDetails',
        id=channel_id
    ).execute()

    uploads_playlist_id = channel_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    videos = []
    next_page_token = None

    while True:
        playlist_response = youtube.playlistItems().list(
            part='snippet,contentDetails',
            playlistId=uploads_playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        for item in playlist_response['items']:
            video_id = item['contentDetails']['videoId']
            videos.append(video_id)

        next_page_token = playlist_response.get('nextPageToken')

        if not next_page_token:
            break

    return videos

video_ids = get_channel_videos(channel_id)


# Function to parse duration
def parse_duration(duration):
    pattern = r'PT(\d+H)?(\d+M)?(\d+S)?'
    match = re.match(pattern, duration)

    hours = int(match.group(1)[:-1]) if match.group(1) else 0
    minutes = int(match.group(2)[:-1]) if match.group(2) else 0
    seconds = int(match.group(3)[:-1]) if match.group(3) else 0

    total_seconds = hours * 3600 + minutes * 60 + seconds
    formatted_time = f"{hours:02}:{minutes:02}:{seconds:02}"

    return total_seconds, formatted_time


# Function to fetch video statistics
def get_video_statistics(video_ids):
    stats = []
    for video_id in video_ids:
        response = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=video_id
        ).execute()

        for item in response['items']:
            duration = item['contentDetails']['duration']
            total_seconds, formatted_time = parse_duration(duration)

            stats.append({
                'videoId': video_id,
                'videoTitle': item['snippet']['title'],
                # 'videoDescription': item['snippet']['description'],
                'datePosted': item['snippet']['publishedAt'],
                'duration': formatted_time,
                'totalSeconds': total_seconds,
                'views': item['statistics']['viewCount'],
                'likes': item['statistics'].get('likeCount', 0),
                'comments': item['statistics'].get('commentCount', 0)
            })
    return stats

video_stats = get_video_statistics(video_ids)
# video_stats

df = pd.DataFrame(video_stats)
df.head()
df.to_csv('iShowSpeedVidData.csv', index=False)